# TextAttack on Keras Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_6_Keras.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_6_Keras.ipynb)

## Training

The code below trains a basic neural network on a series of movie reviews from the IMDB dataset, loaded using Tensorflow's datasets module. Each review is encoded as a sequence of tokens corresponding to a word's index in the vocabulary. Class labels are provided, denoting a positive or negative sentiment. 

See [here](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data) for more information on the IMDB dataset. 


In [1]:
import tensorflow as tf
import keras
import numpy as np
from keras.utils import to_categorical
from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019

import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

from nltk.tokenize import word_tokenize, RegexpTokenizer


Below, we load the IMDB dataset from Tensorflow and transform it for our classifier, using a Bag-of-Words format. 

In [2]:

NUM_WORDS = 1000

(x_train_tokens, y_train), (x_test_tokens, y_test) = tf.keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=NUM_WORDS,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

def transform(x):
  x_transform = []
  for i, word_indices in enumerate(x):
    BoW_array = np.zeros((NUM_WORDS,))
    for index in word_indices:
      if index < len(BoW_array):
        BoW_array[index] += 1
    x_transform.append(BoW_array)
  return np.array(x_transform)
    

index = int(0.9 * len(x_train_tokens))
x_train = transform(x_train_tokens)[:index]
x_test = transform(x_test_tokens)[index:]
y_train = np.array(y_train[:index])
y_test = np.array(y_test[index:])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

vocabulary = tf.keras.datasets.imdb.get_word_index(
    path='imdb_word_index.json'
)

1646592/1641221 [==============================] - 0s 0us/step


With our data successfully loaded, we can now design and trained our model. 

In [3]:
#Model Created with Keras
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=NUM_WORDS))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
opt = keras.optimizers.Adam(learning_rate=0.00001)

model.compile(
 optimizer = opt,
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)


results = model.fit(
 x_train, y_train,
 epochs= 18,
 batch_size = 512,
 validation_data = (x_test, y_test)
)


print(results.history)


Epoch 1/18
44/44 [==============================] - 0s 9ms/step - loss: 0.9584 - accuracy: 0.4987 - val_loss: 0.7314 - val_accuracy: 0.5056
Epoch 2/18
44/44 [==============================] - 0s 6ms/step - loss: 0.9078 - accuracy: 0.5064 - val_loss: 0.7149 - val_accuracy: 0.5332
Epoch 3/18
44/44 [==============================] - 0s 6ms/step - loss: 0.8743 - accuracy: 0.5264 - val_loss: 0.7000 - val_accuracy: 0.5600
Epoch 4/18
44/44 [==============================] - 0s 6ms/step - loss: 0.8534 - accuracy: 0.5385 - val_loss: 0.6840 - val_accuracy: 0.5904
Epoch 5/18
44/44 [==============================] - 0s 6ms/step - loss: 0.8329 - accuracy: 0.5564 - val_loss: 0.6754 - val_accuracy: 0.6064
Epoch 6/18
44/44 [==============================] - 0s 6ms/step - loss: 0.8168 - accuracy: 0.5615 - val_loss: 0.6637 - val_accuracy: 0.6348
Epoch 7/18
44/44 [==============================] - 0s 6ms/step - loss: 0.7942 - accuracy: 0.5767 - val_loss: 0.6568 - val_accuracy: 0.6460
Epoch 8/18
44/44 [==

## Attacking

With our model trained, we can create a  `ModelWrapper` that will allow us to run TextAttack on a custom Keras model. Each `ModelWrapper` must implement a single method, `__call__`, which takes a list of strings and returns a `List`, `np.ndarray`, or `torch.Tensor` of predictions.

In [4]:
class CustomKerasModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
      
      x_transform = []
      for i, review in enumerate(text_input_list):
        tokens = [x.strip(",") for x in review.split()]
        BoW_array = np.zeros((NUM_WORDS,))
        for word in tokens:
          if word in vocabulary:
            if vocabulary[word] < len(BoW_array):
              BoW_array[vocabulary[word]] += 1            
        x_transform.append(BoW_array)
      x_transform = np.array(x_transform)
      prediction = self.model.predict(x_transform)
      return prediction


CustomKerasModelWrapper(model)(["bad bad bad bad bad", "good good good good"])

array([[0.44404104, 0.5262513 ],
       [0.49010894, 0.49974558]], dtype=float32)

With our `ModelWrapper` constructed, we can use TextAttack's HuggingFaceDataset module to load reviews for testing, alongside TextAttack's PWWSRen2019 module to serve as our attack recipe. 

The attack below leverages TextAttack's `Attack` class, capable of running attacks against entire datasets. 


In [5]:
from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack import Attacker

model_wrapper = CustomKerasModelWrapper(model)
dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)

attack = PWWSRen2019.build(model_wrapper)

attack_args = AttackArgs(num_examples=10, checkpoint_dir="checkpoints")

attacker = Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/p/qdata/jy2ma/.cache/textattack/datasets/rotten_tomatoes_movie_review/default/1.0.0/9c411f7ecd9f3045389de0d9ce984061a1056507703d2e3183b1ac1a90816e4d)
textattack: Loading datasets dataset rotten_tomatoes, split test.
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|█         | 1/10 [00:00<00:00, 17.58it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
Negative (50%) --> [SKIPPED]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .




[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:  20%|██        | 2/10 [00:00<00:00,  8.59it/s]

--------------------------------------------- Result 2 ---------------------------------------------
Positive (50%) --> [FAILED]

consistently clever and suspenseful .




[Succeeded / Failed / Skipped / Total] 1 / 1 / 3 / 5:  50%|█████     | 5/10 [00:00<00:00,  5.88it/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
Positive (50%) --> Negative (51%)

fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .

fresnadillo has something serious to tell about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .


--------------------------------------------- Result 7 ---------------------------------------------
Negative (51%) --> [SKIPPED]

throws in enough clever and unexpected twists to make the formula feel fresh .


--------------------------------------------- Result 8 ---------------------------------------------
Negative (51%) --> [SKIPPED]

weighty and ponderous but every bit as filling as the treat of the title .




[Succeeded / Failed / Skipped / Total] 3 / 1 / 5 / 9:  90%|█████████ | 9/10 [00:01<00:00,  4.89it/s]

--------------------------------------------- Result 9 ---------------------------------------------
Positive (50%) --> Negative (50%)

a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .

a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance society office .




[Succeeded / Failed / Skipped / Total] 4 / 1 / 5 / 10: 100%|██████████| 10/10 [00:02<00:00,  4.86it/s]

--------------------------------------------- Result 10 ---------------------------------------------
Positive (51%) --> Negative (50%)

generates an enormous feeling of empathy for its characters .

generates an enormous look of empathy for its characters .



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 4     |
| Number of failed attacks:     | 1     |
| Number of skipped attacks:    | 5     |
| Original accuracy:            | 50.0% |
| Accuracy under attack:        | 10.0% |
| Attack success rate:          | 80.0% |
| Average perturbed word %:     | 7.24% |
| Average num. words per input: | 15.4  |
| Avg num queries:              | 103.2 |
+-------------------------------+-------+


## Newer Keras versions (TF2.5+)
Special considerations need to be taken for newer Keras versions which have been merged inside Tensorflow after TF 2.5 release. One big change from older versions is that Keras models are now non-picklable by the `pickle` package. This prevents performing attacks using `textattack` in parallel mode. To get over this problem a hotfix introduced [here](https://github.com/tensorflow/tensorflow/issues/34697) needs to be employed and subsequent python code needs to be run under a `if __name__ == '__main__'` nest.

In [1]:
# globals()

In [15]:
# Textattack + General imports 
import tensorflow as tf
import keras
import pickle
import numpy as np
import torch
from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import AttackArgs
from textattack import Attacker
from textattack.datasets import Dataset

In [16]:
# Tensorflow 2.5+ (Keras) imports
# Note that keras is now tensorflow.keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

In [17]:
# # This special hotfix to ensure keras models remain picklable
def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

# Hotfix function
def make_keras_picklable():

    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))

    cls = Model
    cls.__reduce__ = __reduce__

# Run the function
make_keras_picklable()

In [18]:
NUM_WORDS = 1000

(x_train_tokens, y_train), (x_test_tokens, y_test) = tf.keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=NUM_WORDS,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

def transform(x):
  x_transform = []
  for i, word_indices in enumerate(x):
    BoW_array = np.zeros((NUM_WORDS,))
    for index in word_indices:
      if index < len(BoW_array):
        BoW_array[index] += 1
    x_transform.append(BoW_array)
  return np.array(x_transform)
    

index = int(0.9 * len(x_train_tokens))
x_train = transform(x_train_tokens)[:index]
x_test = transform(x_test_tokens)[index:]
y_train = np.array(y_train[:index])
y_test = np.array(y_test[index:])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

vocabulary = tf.keras.datasets.imdb.get_word_index(
    path='imdb_word_index.json'
)

In [19]:
#Model Created with Keras
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=NUM_WORDS))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

model.compile(
 optimizer = opt,
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)


results = model.fit(
 x_train, y_train,
 epochs= 18,
 batch_size = 512,
 validation_data = (x_test, y_test)
)


print(results.history)

Epoch 1/18
44/44 [==============================] - 1s 16ms/step - loss: 0.9439 - accuracy: 0.5089 - val_loss: 0.7319 - val_accuracy: 0.5280
Epoch 2/18
44/44 [==============================] - 1s 14ms/step - loss: 0.8806 - accuracy: 0.5183 - val_loss: 0.7149 - val_accuracy: 0.5544
Epoch 3/18
44/44 [==============================] - 1s 11ms/step - loss: 0.8481 - accuracy: 0.5394 - val_loss: 0.6972 - val_accuracy: 0.5812
Epoch 4/18
44/44 [==============================] - 1s 13ms/step - loss: 0.8323 - accuracy: 0.5478 - val_loss: 0.6817 - val_accuracy: 0.6044
Epoch 5/18
44/44 [==============================] - 1s 13ms/step - loss: 0.8171 - accuracy: 0.5541 - val_loss: 0.6687 - val_accuracy: 0.6160
Epoch 6/18
44/44 [==============================] - 1s 12ms/step - loss: 0.7927 - accuracy: 0.5723 - val_loss: 0.6586 - val_accuracy: 0.6324
Epoch 7/18
44/44 [==============================] - 1s 13ms/step - loss: 0.7726 - accuracy: 0.5816 - val_loss: 0.6463 - val_accuracy: 0.6504
Epoch 8/18
44

In [20]:
class CustomKerasModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
      x_transform = []
      for i, review in enumerate(text_input_list):
        tokens = [x.strip(",") for x in review.split()]
        BoW_array = np.zeros((NUM_WORDS,))
        for word in tokens:
          if word in vocabulary:
            if vocabulary[word] < len(BoW_array):
              BoW_array[vocabulary[word]] += 1            
        x_transform.append(BoW_array)
      x_transform = np.array(x_transform)
      prediction = self.model.predict(x_transform)
      return prediction

CustomKerasModelWrapper(model)(["bad bad bad bad bad", "good good good good"])

array([[0.48411477, 0.50139904],
       [0.48761007, 0.4937562 ]], dtype=float32)

In [ ]:
# locals()['CustomKerasModelWrapper']
# globals()['CustomKerasModelWrapper']
# globals()['__builtin__']

In [ ]:
# # global CustomKerasModelWrapper
# # CustomKerasModelWrapper = locals()['CustomKerasModelWrapper']
# # globals()['CustomKerasModelWrapper']

# # import __builtin__
# # globals()['__builtin__'].CustomKerasModelWrapper = CustomKerasModelWrapper

# import sys

# def add_submodule(submod, parentname):
#     name = submod.__name__
#     sys.modules[parentname + "." + name] = submod(name)
    
# add_submodule(CustomKerasModelWrapper,'__name__')

In [ ]:
# print(__name__)
# globals()['CustomKerasModelWrapper'] = CustomKerasModelWrapper

In [21]:
torch.multiprocessing.freeze_support()

model_wrapper = CustomKerasModelWrapper(model)

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)

attack = PWWSRen2019.build(model_wrapper)

attack_args = AttackArgs(num_examples=10, checkpoint_dir="checkpoints", parallel=True, num_workers_per_device=2)

attacker = Attacker(attack, dataset, attack_args)

attacker.attack_dataset()


Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/u/ss7mu/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a)
textattack: Loading datasets dataset rotten_tomatoes, split test.
Loading cached shuffled indices for dataset at /u/ss7mu/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a/cache-61155c511b20c4eb.arrow
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 8 worker(s) on 4 GPU(s).
2021-08-13 10:47:27.450192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file

KeyboardInterrupt: 

## Conclusion

Great! We trained a binary classifier, created a custom `ModelWrapper` for Keras models, and successsfully ran adversarial attacks against our trained Keras model! This serves a basic demo for how to use TextAttack within your own environments. 
